In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Installing dependecies

In [ ]:
!pip install ultralytics

In [ ]:
!pip install -U ipywidgets

In [ ]:
!pip install wandb

#### Importing libraries

In [ ]:
# Import necessary libraries
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import wandb

In [ ]:
# Define base paths
base_dataset_path = '/kaggle/input/idd-dataset/IDD_Detection_Organized'
output_base_path = '/kaggle/working/IDD_Detection_YOLO'
subsets = ['train', 'val']

# Define the classes and their indices
classes = {
    'person': 0,
    'rider': 1,
    'motorcycle': 2,
    'bicycle': 3,
    'autorickshaw': 4,
    'car': 5,
    'truck': 6,
    'bus': 7,
    'traffic light': 8,
    'traffic sign': 9
}

# Classes to ignore during label update
classes_to_ignore = {'traffic sign', 'bicycle', 'traffic light'}
indices_to_ignore = {classes[class_name] for class_name in classes_to_ignore}

#### Utils

In [ ]:
def copy_images(source_dir, destination_dir):
    """Copy images from source to destination directory."""
    os.makedirs(destination_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        if filename.endswith(('.jpg', '.png')):
            shutil.copy2(os.path.join(source_dir, filename), os.path.join(destination_dir, filename))
    print(f"All images from {source_dir} have been copied to {destination_dir}.")

def convert_annotation_to_yolo_format(xml_file, img_width, img_height):
    """Convert XML annotations to YOLO format."""
    yolo_annotations = []
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for obj in root.findall('object'):
        cls_name = obj.find('name').text
        if cls_name not in classes:
            continue  # Skip if class is not in our predefined classes

        cls_id = classes[cls_name]

        xmlbox = obj.find('bndbox')
        xmin = int(xmlbox.find('xmin').text)
        ymin = int(xmlbox.find('ymin').text)
        xmax = int(xmlbox.find('xmax').text)
        ymax = int(xmlbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2.0) / img_width
        y_center = ((ymin + ymax) / 2.0) / img_height
        width = (xmax - xmin) / float(img_width)
        height = (ymax - ymin) / float(img_height)

        yolo_annotations.append((cls_id, x_center, y_center, width, height))
    
    return yolo_annotations

def save_yolo_annotations(xml_file, txt_file, img_width, img_height):
    """Convert XML annotations to YOLO format and save them to a text file."""
    yolo_annotations = convert_annotation_to_yolo_format(xml_file, img_width, img_height)
    with open(txt_file, 'w') as f:
        for ann in yolo_annotations:
            cls_id, x_center, y_center, width, height = ann
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def process_annotations(subset):
    """Process annotations and images for a given subset (train/val)."""
    images_path = os.path.join(base_dataset_path, subset, 'images')
    annotations_path = os.path.join(base_dataset_path, subset, 'annotations')
    yolo_labels_path = os.path.join(output_base_path, subset, 'labels')

    os.makedirs(yolo_labels_path, exist_ok=True)

    for annotation_file in os.listdir(annotations_path):
        if annotation_file.endswith('.xml'):
            xml_file = os.path.join(annotations_path, annotation_file)
            image_file = os.path.join(images_path, annotation_file.replace('.xml', '.jpg'))
            txt_file = os.path.join(yolo_labels_path, annotation_file.replace('.xml', '.txt'))

            if not os.path.exists(image_file):
                print(f"Image file not found for annotation: {annotation_file}")
                continue

            img = cv2.imread(image_file)
            img_height, img_width, _ = img.shape
            save_yolo_annotations(xml_file, txt_file, img_width, img_height)

    print(f"All annotations for {subset} have been converted to YOLO format successfully.")

def remove_empty_labels(labels_path):
    """Remove empty or corrupted label files."""
    empty_labels = [f for f in os.listdir(labels_path) if os.path.getsize(os.path.join(labels_path, f)) == 0]
    for label_file in empty_labels:
        os.remove(os.path.join(labels_path, label_file))
        print(f"Removed corrupted label file: {label_file}")

def update_labels(labels_path):
    """Update label files to exclude specified classes."""
    for label_file in os.listdir(labels_path):
        if label_file.endswith('.txt'):
            file_path = os.path.join(labels_path, label_file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            
            updated_lines = [line for line in lines if int(line.split()[0]) not in indices_to_ignore]
            
            with open(file_path, 'w') as f:
                f.writelines(updated_lines)

def create_data_yaml():
    """Create the data.yaml file for YOLO training."""
    data_yaml_content = f"""
train: {os.path.join(output_base_path, 'train/images')}
val: {os.path.join(output_base_path, 'val/images')}

nc: {len(classes)}  # Number of classes
names: {list(classes.keys())}
"""
    with open('/kaggle/working/data.yaml', 'w') as file:
        file.write(data_yaml_content)
    print("data.yaml file created successfully!")


In [ ]:
# Copy images and process annotations for each subset
for subset in subsets:
    copy_images(os.path.join(base_dataset_path, subset, 'images'), os.path.join(output_base_path, subset, 'images'))
    process_annotations(subset)
    remove_empty_labels(os.path.join(output_base_path, subset, 'labels'))
    update_labels(os.path.join(output_base_path, subset, 'labels'))

#### YAML

In [ ]:
create_data_yaml()

In [ ]:
# Initialize Weights & Biases
wandb.init(project='IDD', name='yolov8n-basemodel-1')

#### Training

In [ ]:
def train_yolo_model():
    """Train the YOLO model with customized hyperparameters."""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    model = YOLO('yolov8n.pt')  # Load the pre-trained YOLOv8n model

    # Set your custom hyperparameters
    results = model.train(
        data='/kaggle/working/data.yaml', 
        epochs=100,  
        batch=16,  
        imgsz=640,  
        lr0=0.001,  
        optimizer='Adam',  
        momentum=0.937,
        weight_decay=0.0005,
        name='yolov8n_idd_basemodel-1',  
        device=device,  
        project='IDD', 
    )
    
    # Save the trained model
    model.save('/kaggle/working/yolov8n_idd_basemodel-1.pt')

    # Validate the model and print key metrics
    val_results = model.val()
    print(f"Validation mAP50: {val_results.box.map50:.4f}, Precision: {val_results.box.mp:.4f}, Recall: {val_results.box.mr:.4f}")

In [ ]:
# Train the model
train_yolo_model()

In [ ]:
# Load the saved YOLO model from the specified path
model = YOLO('/kaggle/working/yolov8n_idd_basemodel.pt')
print("Model loaded successfully.")

#### Testing

In [ ]:
# Test on a single image
test_image_path = '/kaggle/input/idd-dataset/IDD_Detection_Organized/val/images/0000285.jpg'

# Run inference on the test image using the loaded model
results = model(test_image_path)

# Function to display image with bounding boxes
def display_results(image_path, results):
    """Display image with YOLO detection results."""
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape

    # Loop through each detection in results
    for result in results[0].boxes:
        cls_id = int(result.cls.item())  # Convert class ID tensor to int
        conf = result.conf.item()  # Convert confidence tensor to float
        xmin, ymin, xmax, ymax = map(int, result.xyxy[0].tolist())  # Convert bounding box coordinates to integers

        # Draw the bounding box
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        # Put class name and confidence on the image
        label = f"{list(classes.keys())[cls_id]}: {conf:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Convert BGR image (OpenCV default) to RGB for matplotlib display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 6))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

# Display the results
display_results(test_image_path, results)